## Урок 3
1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД<br>
2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы<br>
3)*Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [2]:
import requests
import json
import pandas as pd
import time
import sys
import re

from pymongo import MongoClient
from bs4 import BeautifulSoup as bs
from requests.exceptions import HTTPError

In [351]:
PATH_PARAMS = "../resource/params.txt"
URL_HH = "https://hh.ru"
URL_SUPERJOB = "https://www.superjob.ru"
USER_AGENT = ""

### Чтение настроек

In [352]:
def get_user_agent():
    with open(PATH_PARAMS, "r") as f:
        USER_AGENT = json.load(f)["user-agent"]

### Подключение к БД

In [353]:
def db_connect():
    return MongoClient('localhost', 27017)['jobs']

### Получение HTML кода по ссылке

In [354]:
def get_html(url):
    try:
        response = requests.get(url = url, headers = {"User-Agent": USER_AGENT})
        if response.ok == True:
            return response.text
        else:
            response.raise_for_status()
    except Exception as err:
        print(f'Error: {err}')
        return ''

### Вывод статуса обработки

In [355]:
def show_status(text, sleep):
    sys.stdout.write(text)
    sys.stdout.flush()
    time.sleep(sleep)

### Чтение HTML с возможностью кэширования

In [356]:
def get_html_cache(use_cache, cache, link):
    sleep = 0
    html = ''
    
    #Если считать данные из кеша не получилось или кэширование выключено...
    if use_cache:
        f = cache.find_one({"link": link})
        if f:
            html = f["html"]
    
    #...то достаём данные с сайта
    if not html:
        html = get_html(link)
        if html:
            cache.insert_one({"link": link, 
                              "html": html})
        sleep = 1
        
    return(html, sleep)

### Парсинг зарплаты

In [357]:
def parse_salary(salary):
    regex = re.search('^(от\s|до\s)?([\d\s]+)([-—][\d\s]+)?\s(\D+)$', salary)

    low = regex.group(2)
    if low:
        low = low.replace(" ", "").replace(u"\u00A0", "") 
    else:
        low = ""
    
    high = regex.group(3)
    if high:
        high = high.replace(" ", "").replace(u"\u00A0", "") .replace("-", "").replace("—", "")
    else:
        high = ""

    if not regex.group(1) and not high:
        high = low

    currency = regex.group(4)

    return (low, high, currency)

### Обновление строки в БД

In [358]:
def update_db(co, vacancy_link, vacancy_name, employer, salary_low, salary_high, currency, main_url):
    co.update_one({'link': vacancy_link}, 
                  {'$set': {'link': vacancy_link,
                            'data': {'vacancy': vacancy_name,
                                     'employer': employer,
                                     'salary_low': int(salary_low) if salary_low else None,
                                     'salary_high': int(salary_high) if salary_high else None,
                                     'currency': currency,
                                     'main_url': main_url
                                    }
                           }
                  },
                  upsert = True
                 )

### Парсинг hh.ru

In [359]:
def parse_hh(vacancy_search, use_cache, db):
    #Инициализация
    link = f'{URL_HH}/search/vacancy?clusters=true&enable_snippets=true&text={vacancy_search}&showClusters=false'
    page = 0
    cache = db.cache
    hh = db.hh

    #Цикл по страницам
    while link:
        html, sleep = get_html_cache(use_cache, cache, link)

        if html:
            parsed_html = bs(html, 'lxml')

            div_vacancies = parsed_html.find('div', {'class':'vacancy-serp'}).find_all(
                'div', {'class':'vacancy-serp-item'})
            
            #Цикл по вакансиям
            for div_vacancy in div_vacancies:
                data = {}
                try:
                    div_vacancy_name = div_vacancy.find('a', {'class':'bloko-link HH-LinkModifier'})
                    vacancy_name = div_vacancy_name.getText()
                    vacancy_link = div_vacancy_name['href']
                except Exception as e:
                    vacancy_name = ''
                    vacancy_link = ''

                try:
                    salary_low, salary_high, currency = parse_salary(
                        div_vacancy.find('div', {'class':'vacancy-serp-item__compensation'}).getText())
                except Exception as e:
                    salary_low = ''
                    salary_high = ''
                    currency = ''

                try:
                    employer = div_vacancy.find('a', {'class':'bloko-link bloko-link_secondary HH-AnonymousIndexAnalytics-Recommended-Company'}).getText()
                except Exception as e:
                    employer = ''
                
                update_db(hh, vacancy_link, vacancy_name, employer, salary_low, salary_high, currency, URL_HH)

            #Ссылка на следующую страницу берётся из кнопки "Дальше"
            try:
                link = URL_HH + \
                    parsed_html.find('a', {'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})['href']
            except Exception as e:
                link = ''

            page += 1
            show_status(f"\rОбработано страниц hh.ru: {page}", sleep)
        else:
            #В случае возникновения ошибки при чтении html выводим сообщение об ошибке
            print(html)
            break

    show_status("\n", 0)

### Парсинг superjob.ru

In [360]:
def parse_superjob(vacancy_search, use_cache, db):
    #Инициализация
    link = f'{URL_SUPERJOB}/vacancy/search/?keywords={vacancy_search}&geo%5Bc%5D%5B0%5D=1'
    page = 0
    cache = db.cache
    sj = db.sj
    
    #Цикл по страницам
    while link:
        html, sleep = get_html_cache(use_cache, cache, link)

        if html:
            parsed_html = bs(html, 'lxml')

            block = parsed_html.find('div', {'class':'_1ID8B'})
            if block:
                div_vacancies = block.find_all('div', {'class':'_3syPg _3P0J7 _9_FPy'})

                #Цикл по вакансиям
                for div_vacancy in div_vacancies:
                    data = {}
                    try:
                        vacancy_name = div_vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).getText()
                    except Exception as e:
                        vacancy_name = ''

                    try:
                        vacancy_link = div_vacancy.find('a', {'class': re.compile("icMQ_ _1QIBo.*_2JivQ _3dPok")})['href']
                    except Exception as e:
                        vacancy_link = ''

                    try:
                        salary_low, salary_high, currency = parse_salary(
                            div_vacancy.find('span', {'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).getText())
                    except Exception as e:
                        salary_low = ''
                        salary_high = ''
                        currency = ''

                    try:
                        employer = div_vacancy.find('a', {'class': re.compile("icMQ_ _205Zx.*Vm5jz")}).getText()
                    except Exception as e:
                        employer = ''

                    update_db(sj, vacancy_link, vacancy_name, employer, salary_low, salary_high, currency, URL_SUPERJOB)

            #Ссылка на следующую страницу берётся из кнопки "Дальше"
            try:
                link = URL_SUPERJOB + \
                    parsed_html.find('a', {'class':'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-dalshe'})['href']
            except Exception as e:
                link = ''

            page += 1
            show_status(f"\rОбработано страниц superjob.ru: {page}", sleep)
        else:
            #В случае возникновения ошибки при чтении html выводим сообщение об ошибке
            print(html)
            break

    show_status("\n", 0)

### Форматирование строки перед выводом

In [364]:
def format_row(f):
    return ['<a href="{}">{}</a>'.format(f['link'], f['data']['vacancy']),
            f['data']['employer'],
            f['data']['salary_low'] if f['data']['salary_low'] else '',
            f['data']['salary_high'] if f['data']['salary_high'] else '',
            f['data']['currency'],
            '<a href="{}">{}</a>'.format(f['data']['main_url'], f['data']['main_url'])
           ]


### Поиск вакансий в БД с минимальной зарплатой

In [365]:
def find_vacancies(min_salary, db):
    #Ищем вакансии у которых минимальная или максимальная зарплата больше либо равна искомой
    find = {'$or': [{'data.salary_low': {'$gte': min_salary}}, 
                    {'data.salary_high': {'$gte': min_salary}}]}    
    
    all_vacancies = []
    
    for f in db.hh.find(find):
        all_vacancies.append(format_row(f))
        
    for f in db.sj.find(find):
        all_vacancies.append(format_row(f))

    return pd.DataFrame(all_vacancies,
                        columns = ['vacancy', 
                                   'employer',
                                   'salary_low', 
                                   'salary_high',
                                   'currency',
                                   'main_url'
                                  ]
                       )

### Сбор информации о вакансиях

In [363]:
#Запрос данных
vacancy = input('Название вакансии: ')

if vacancy:
    #Приоритет чтения информации из кэша, а не с сайта
    use_cache = True 
        
    #Чтение настроек
    get_user_agent()
    
    #Подключаемся к БД
    db = db_connect()

    #Парсинг hh.ru
    data = parse_hh(vacancy, use_cache, db)
    
    #Парсинг superjob.ru
    data = parse_superjob(vacancy, use_cache, db)

Название вакансии: Python
Обработано страниц hh.ru: 100
Обработано страниц superjob.ru: 5


### Поиск вакансий по зарплате

In [367]:
#Запрос данных
min_salary = int(input('Минимальная зарплата: '))

#Ищем искомые вакансии
all_vacancies = find_vacancies(min_salary, db_connect())

all_vacancies.style

Минимальная зарплата: 200000


,vacancy,employer,salary_low,salary_high,currency,main_url
0,Python разработчик,ТОО TargetAI Limited,400000,600000,KZT,https://hh.ru
1,Kubernetes Engineer,ООО Рекон Групп,200000,,руб.,https://hh.ru
2,Ведущий разработчик Python / Software development / Team lead,CATAPULTO.RU,200000,,руб.,https://hh.ru
3,Senior PHP Developer / PHP разработчик,Сервисный центр Apple,160000,220000,руб.,https://hh.ru
4,Программист Python,"Insilico Medicine, Inc",120000,220000,руб.,https://hh.ru
5,Python developer,IVIDEON,160000,200000,руб.,https://hh.ru
6,Python developer,B2Broker Санкт-Петербург,150000,200000,руб.,https://hh.ru
7,Python разработчик / Python software developer,Cindicator,400000,,руб.,https://hh.ru
8,Python разработчик / Python software developer,Cindicator,400000,,руб.,https://hh.ru
9,Ведущий программист Python,ООО Эргиус,300000,,руб.,https://hh.ru
